## Statemachine test

In [14]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
# import dmyplant2
# dmyplant2.cred()
# mp = dmyplant2.MyPlant(600)
# e=dmyplant2.Engine.from_sn(mp,1184199)
#print(f"{e}")
#e.dash
#dtrips = e.batch_hist_alarms(p_severities=[600,650,700,800], p_offset=0, p_limit=200000)
#dtrips.to_pickle('1184199_messages.pkl')

In [31]:
from dfsm import msgFSM
messages = pd.read_pickle('1184199_messages.pkl')
messages.shape

(127515, 8)

In [33]:
from dfsm import regexFSM, grep_regex
grep_regex("abc")

S  -> Q1
Q1 -> Q2
Q2 -> Q3


True